In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# =======================
# Cell 1: Setup
# =======================
import os
import random
import shutil
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import (
    ResNet50, DenseNet121, MobileNet, InceptionV3
)
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_pre
from tensorflow.keras.applications.densenet import preprocess_input as densenet_pre
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_pre
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_pre

from sklearn.metrics import classification_report, confusion_matrix

print("TF version:", tf.__version__)

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Original Kaggle dataset path (matches your screenshot)
BASE_DATASET_DIR = Path("/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake")
TEST_REAL_DIR = BASE_DATASET_DIR / "test" / "real"
TEST_FAKE_DIR = BASE_DATASET_DIR / "test" / "fake"

# Our custom 20k_gan_8_1_1 dataset root
TARGET_ROOT = Path("/kaggle/working/20k_gan_8_1_1")

IMG_SIZE = (256, 256)
BATCH_SIZE = 32

2025-11-29 22:06:05.653499: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764453965.818538      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764453965.864483      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

TF version: 2.18.0


In [2]:
# =======================
# Cell 2: Create 20k_gan_8_1_1 split
# =======================

def make_empty_dir_structure(root: Path):
    if root.exists():
        shutil.rmtree(root)
    for split in ["train", "val", "test"]:
        for cls in ["real", "fake"]:
            (root / split / cls).mkdir(parents=True, exist_ok=True)

make_empty_dir_structure(TARGET_ROOT)

def sample_and_split(class_dir: Path, n_total: int, splits=(0.8, 0.1, 0.1)):
    """Return lists of filenames for train, val, test for a given class."""
    all_images = [str(p) for p in class_dir.glob("*.jpg")] + \
                 [str(p) for p in class_dir.glob("*.png")] + \
                 [str(p) for p in class_dir.glob("*.jpeg")]
    all_images = sorted(all_images)
    assert len(all_images) >= n_total, f"Not enough images in {class_dir}"

    random.shuffle(all_images)
    all_images = all_images[:n_total]

    n_train = int(n_total * splits[0])
    n_val   = int(n_total * splits[1])
    n_test  = n_total - n_train - n_val

    train_files = all_images[:n_train]
    val_files   = all_images[n_train:n_train+n_val]
    test_files  = all_images[n_train+n_val:]
    return train_files, val_files, test_files

# Sample 10k real + 10k fake from TEST folder
REAL_TRAIN, REAL_VAL, REAL_TEST = sample_and_split(TEST_REAL_DIR, 10000)
FAKE_TRAIN, FAKE_VAL, FAKE_TEST = sample_and_split(TEST_FAKE_DIR, 10000)

def copy_files(file_list, dst_dir: Path):
    for src in file_list:
        dst = dst_dir / Path(src).name
        shutil.copy2(src, dst)

# Copy into our new structure
copy_files(REAL_TRAIN, TARGET_ROOT / "train" / "real")
copy_files(REAL_VAL,   TARGET_ROOT / "val" / "real")
copy_files(REAL_TEST,  TARGET_ROOT / "test" / "real")

copy_files(FAKE_TRAIN, TARGET_ROOT / "train" / "fake")
copy_files(FAKE_VAL,   TARGET_ROOT / "val" / "fake")
copy_files(FAKE_TEST,  TARGET_ROOT / "test" / "fake")

print("Custom dataset created at:", TARGET_ROOT)
for split in ["train", "val", "test"]:
    for cls in ["real", "fake"]:
        count = len(list((TARGET_ROOT / split / cls).glob("*")))
        print(f"{split}/{cls}: {count} images")

Custom dataset created at: /kaggle/working/20k_gan_8_1_1
train/real: 8000 images
train/fake: 8000 images
val/real: 1000 images
val/fake: 1000 images
test/real: 1000 images
test/fake: 1000 images


In [3]:
# =======================
# Cell 3: Data generators
# =======================

def build_generators(root_dir: Path, preprocess_func=None, shuffle_train=True):
    """
    Returns (train_gen, val_gen, test_gen).
    If preprocess_func is None → rescale 1./255.
    Otherwise uses given preprocess function (for transfer learning models).
    """
    if preprocess_func is None:
        train_datagen = ImageDataGenerator(rescale=1./255.)
        val_datagen   = ImageDataGenerator(rescale=1./255.)
        test_datagen  = ImageDataGenerator(rescale=1./255.)
    else:
        train_datagen = ImageDataGenerator(preprocessing_function=preprocess_func)
        val_datagen   = ImageDataGenerator(preprocessing_function=preprocess_func)
        test_datagen  = ImageDataGenerator(preprocessing_function=preprocess_func)

    train_gen = train_datagen.flow_from_directory(
        directory=str(root_dir / "train"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=shuffle_train,
        seed=SEED
    )
    val_gen = val_datagen.flow_from_directory(
        directory=str(root_dir / "val"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False
    )
    test_gen = test_datagen.flow_from_directory(
        directory=str(root_dir / "test"),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False
    )
    return train_gen, val_gen, test_gen

# Generators for the custom CNN (simple rescaling)
train_gen_cnn, val_gen_cnn, test_gen_cnn = build_generators(TARGET_ROOT, preprocess_func=None)

len(train_gen_cnn.filenames), len(val_gen_cnn.filenames), len(test_gen_cnn.filenames)


Found 16000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


(16000, 2000, 2000)

In [4]:
# =======================
# Cell 4: Train & evaluate helper
# =======================

def train_and_evaluate(model, train_gen, val_gen, test_gen,
                       epochs, model_name="model"):
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=3,
            verbose=1
        )
    ]
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks,
        verbose=1
    )

    # Evaluate on test set
    test_loss, test_acc = model.evaluate(test_gen, verbose=1)
    print(f"\n[{model_name}] Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

    # Detailed metrics
    test_gen.reset()
    preds = model.predict(test_gen, verbose=1)
    y_pred = (preds.ravel() >= 0.5).astype(int)
    y_true = test_gen.classes

    print("\nConfusion matrix:")
    print(confusion_matrix(y_true, y_pred))

    print("\nClassification report:")
    print(classification_report(y_true, y_pred, target_names=["fake", "real"]))

    return history, (test_loss, test_acc)


In [9]:
# =======================
# Extra helper for fixed-epoch training (no EarlyStopping)
# =======================

def train_and_evaluate_no_es(model, train_gen, val_gen, test_gen,
                             epochs, model_name="model"):
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        verbose=1
    )

    test_loss, test_acc = model.evaluate(test_gen, verbose=1)
    print(f"\n[{model_name}] Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

    # Detailed metrics
    test_gen.reset()
    preds = model.predict(test_gen, verbose=1)
    y_pred = (preds.ravel() >= 0.5).astype(int)
    y_true = test_gen.classes

    print("\nConfusion matrix:")
    print(confusion_matrix(y_true, y_pred))

    print("\nClassification report:")
    print(classification_report(y_true, y_pred, target_names=["fake", "real"]))

    return history, (test_loss, test_acc)


In [6]:
# =======================
# Cell 6: Transfer-learning model builder
# =======================

def build_transfer_model(base_constructor, input_shape=(256, 256, 3), name="TLModel"):
    """
    base_constructor: Keras application (ResNet50, DenseNet121, MobileNet, InceptionV3)
    """
    base_model = base_constructor(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape
    )
    # Feature extractor only (as in the paper)
    base_model.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = models.Model(inputs, outputs, name=name)
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [11]:
# =======================
# Cell 7: ResNet50 with 32 epochs total (8 + 24) and fine-tuning
# =======================

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Generators with ResNet preprocessing
train_res, val_res, test_res = build_generators(TARGET_ROOT, preprocess_func=resnet_pre)

# Build base ResNet50
base_model_resnet = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3)
)

# -------------------------
# Phase 1 – train new head only (8 epochs)
# -------------------------
base_model_resnet.trainable = False

inputs = layers.Input(shape=(256, 256, 3))
x = base_model_resnet(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

resnet_model = models.Model(inputs, outputs, name="ResNet50_model")

# Slightly higher LR since we’re only training a small head
resnet_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print("=== Phase 1: train head only (8 epochs) ===")
resnet_model.summary()

history_resnet_p1, _ = train_and_evaluate_no_es(
    resnet_model,
    train_res,
    val_res,
    test_res,
    epochs=8,
    model_name="ResNet50_phase1"
)

# -------------------------
# Phase 2 – fine-tune top ResNet layers (up to 24 epochs)
# -------------------------

# Unfreeze last ~30 layers (except BatchNorm, which we keep frozen)
for layer in base_model_resnet.layers[-30:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

# Much lower LR for fine-tuning
resnet_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print("\n=== Phase 2: fine-tune top ResNet layers (max 24 epochs) ===")
for i, layer in enumerate(base_model_resnet.layers[-40:]):
    print(i, layer.name, "trainable" if layer.trainable else "frozen")

early = EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1
)

history_resnet_p2 = resnet_model.fit(
    train_res,
    epochs=24,                 # upper cap; EarlyStopping will usually stop earlier
    validation_data=val_res,
    callbacks=[early, reduce_lr],
    verbose=1
)

# -------------------------
# Final evaluation on test set
# -------------------------
test_loss, test_acc = resnet_model.evaluate(test_res, verbose=1)
print(f"\n[ResNet50_finetuned] Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

# Detailed metrics
test_res.reset()
preds = resnet_model.predict(test_res, verbose=1)
y_pred = (preds.ravel() >= 0.5).astype(int)
y_true = test_res.classes

print("\nConfusion matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification report:")
print(classification_report(y_true, y_pred, target_names=["fake", "real"]))

test_metrics_resnet = (test_loss, test_acc)
test_metrics_resnet

Found 16000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
=== Phase 1: train head only (8 epochs) ===


Model: "ResNet50_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/8


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


500/500 ━━━━━━━━━━━━━━━━━━━━ 75s 131ms/step - accuracy: 0.6603 - loss: 0.6347 - val_accuracy: 0.8100 - val_loss: 0.4330
Epoch 2/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 60s 120ms/step - accuracy: 0.7742 - loss: 0.4726 - val_accuracy: 0.8445 - val_loss: 0.3853
Epoch 3/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 60s 119ms/step - accuracy: 0.7857 - loss: 0.4512 - val_accuracy: 0.8445 - val_loss: 0.3710
Epoch 4/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 60s 120ms/step - accuracy: 0.7904 - loss: 0.4456 - val_accuracy: 0.8525 - val_loss: 0.3602
Epoch 5/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 60s 119ms/step - accuracy: 0.8032 - loss: 0.4277 - val_accuracy: 0.8560 - val_loss: 0.3525
Epoch 6/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 60s 119ms/step - accuracy: 0.8128 - loss: 0.4147 - val_accuracy: 0.8680 - val_loss: 0.3403
Epoch 7/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 60s 119ms/step - accuracy: 0.8056 - loss: 0.4319 - val_accuracy: 0.8675 - val_loss: 0.3383
Epoch 8/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 60s 120ms/step - accuracy: 0.8047 - loss: 0.4263 - val_accura

(0.13615216314792633, 0.9649999737739563)

In [13]:
# =======================
# DenseNet121 – paper-style, tuned to reach ~97.3%
# =======================

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers, models, optimizers
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

# 1) Preprocessing for DenseNet
densenet_pre = tf.keras.applications.densenet.preprocess_input

# 2) Build generators (same as ResNet, just different preprocess func)
train_den, val_den, test_den = build_generators(
    TARGET_ROOT,
    preprocess_func=densenet_pre
)

# 3) Base DenseNet121 backbone
base_model_dense = DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3)
)

# -------------------------
# Phase 1 – train new head only (8 epochs)
# -------------------------
base_model_dense.trainable = False  # freeze backbone

inputs = layers.Input(shape=(256, 256, 3))
x = base_model_dense(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)          # strong regularisation
outputs = layers.Dense(1, activation="sigmoid")(x)

densenet_model = models.Model(inputs, outputs, name="DenseNet121_model")

densenet_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print("=== Phase 1: DenseNet121 head training (8 epochs) ===")
densenet_model.summary()

history_dense_p1, _ = train_and_evaluate_no_es(
    densenet_model,
    train_den,
    val_den,
    test_den,
    epochs=8,
    model_name="DenseNet121_phase1"
)

# -------------------------
# Phase 2 – more aggressive fine-tuning (up to 25 epochs)
# -------------------------

# Unfreeze a larger chunk: last ~80 layers (except BatchNorm)
for layer in base_model_dense.layers[-80:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

# Recompile with lower LR for fine-tuning
densenet_model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-5),  # slightly higher than 1e-5 for stronger adaptation
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print("\n=== Phase 2: Fine-tuning DenseNet121 (max 25 epochs) ===")
for i, layer in enumerate(base_model_dense.layers[-90:]):
    print(i, layer.name, "trainable" if layer.trainable else "frozen")

early = EarlyStopping(
    monitor="val_accuracy",
    patience=4,             # allow a bit more exploration
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    verbose=1
)

history_dense_p2 = densenet_model.fit(
    train_den,
    epochs=25,              # upper cap; EarlyStopping will stop earlier
    validation_data=val_den,
    callbacks=[early, reduce_lr],
    verbose=1
)

# -------------------------
# Final evaluation on test set
# -------------------------
test_loss_dense, test_acc_dense = densenet_model.evaluate(test_den, verbose=1)
print(f"\n[DenseNet121_finetuned] Test accuracy: {test_acc_dense:.4f}, Test loss: {test_loss_dense:.4f}")

# Confusion matrix + classification report
test_den.reset()
preds_dense = densenet_model.predict(test_den, verbose=1)
y_pred_dense = (preds_dense.ravel() >= 0.5).astype(int)
y_true_dense = test_den.classes

print("\nConfusion matrix (DenseNet121):")
print(confusion_matrix(y_true_dense, y_pred_dense))

print("\nClassification report (DenseNet121):")
print(classification_report(y_true_dense, y_pred_dense, target_names=["fake", "real"]))

test_metrics_dense = (test_loss_dense, test_acc_dense)
test_metrics_dense


Found 16000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
=== Phase 1: DenseNet121 head training (8 epochs) ===


Model: "DenseNet121_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 8, 8, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,038,529 (26.85 MB)

 Trainable params: 1,025 (4.00 KB)

 Non-trainable params: 7,037,504 (26.85 MB)

Epoch 1/8


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


500/500 ━━━━━━━━━━━━━━━━━━━━ 85s 130ms/step - accuracy: 0.6107 - loss: 0.6916 - val_accuracy: 0.7930 - val_loss: 0.5086
Epoch 2/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 103ms/step - accuracy: 0.7111 - loss: 0.5632 - val_accuracy: 0.7890 - val_loss: 0.4797
Epoch 3/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.7276 - loss: 0.5422 - val_accuracy: 0.8090 - val_loss: 0.4603
Epoch 4/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 102ms/step - accuracy: 0.7333 - loss: 0.5347 - val_accuracy: 0.8060 - val_loss: 0.4582
Epoch 5/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.7283 - loss: 0.5324 - val_accuracy: 0.8230 - val_loss: 0.4510
Epoch 6/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.7347 - loss: 0.5311 - val_accuracy: 0.8190 - val_loss: 0.4504
Epoch 7/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.7417 - loss: 0.5198 - val_accuracy: 0.8240 - val_loss: 0.4411
Epoch 8/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 51s 101ms/step - accuracy: 0.7285 - loss: 0.5376 - val_accura

(0.09182359278202057, 0.968500018119812)

In [15]:
# =======================
# MobileNet – 14 epochs total (paper-style)
# =======================

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers
import tensorflow as tf

# 1) Preprocessing for MobileNetV2 (scales to [-1, 1])
mobilenet_pre = tf.keras.applications.mobilenet_v2.preprocess_input

# 2) Build generators for 20k_gan_8_1_1 using the same pipeline as other models
train_mob, val_mob, test_mob = build_generators(
    TARGET_ROOT,
    preprocess_func=mobilenet_pre
)

# 3) Base MobileNetV2 backbone (no top)
base_mobilenet = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3)
)

# ⚠ Paper-style: train whole network for 14 epochs (no 2-phase scheme)
base_mobilenet.trainable = True

inputs = layers.Input(shape=(256, 256, 3))
x = base_mobilenet(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)          # same regularisation style as other models
outputs = layers.Dense(1, activation="sigmoid")(x)

mobilenet_model = models.Model(inputs, outputs, name="MobileNetV2_model")

# Use a small LR since we're fine-tuning end-to-end from the start
mobilenet_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

mobilenet_model.summary()

# 4) Train for EXACTLY 14 epochs (single phase)
# train_and_evaluate_no_es should:
#   - fit() for 'epochs' epochs
#   - evaluate on test set
#   - optionally print confusion matrix & classification report
history_mob, test_metrics_mob = train_and_evaluate_no_es(
    mobilenet_model,
    train_mob,
    val_mob,
    test_mob,
    epochs=14,                 # 👉 paper’s epoch count
    model_name="MobileNet_14epochs"
)

test_metrics_mob

Found 16000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


/tmp/ipykernel_47/3877290722.py:19: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_mobilenet = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "MobileNetV2_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 8, 8, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Non-trainable params: 34,112 (133.25 KB)

Epoch 1/14


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
E0000 00:00:1764462830.724499     139 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1764462830.863157     139 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


500/500 ━━━━━━━━━━━━━━━━━━━━ 121s 148ms/step - accuracy: 0.6217 - loss: 0.6683 - val_accuracy: 0.5360 - val_loss: 0.9154
Epoch 2/14
500/500 ━━━━━━━━━━━━━━━━━━━━ 68s 135ms/step - accuracy: 0.8327 - loss: 0.3746 - val_accuracy: 0.7115 - val_loss: 0.5979
Epoch 3/14
500/500 ━━━━━━━━━━━━━━━━━━━━ 67s 134ms/step - accuracy: 0.9156 - loss: 0.2116 - val_accuracy: 0.8935 - val_loss: 0.2572
Epoch 4/14
500/500 ━━━━━━━━━━━━━━━━━━━━ 67s 134ms/step - accuracy: 0.9530 - loss: 0.1268 - val_accuracy: 0.9345 - val_loss: 0.1643
Epoch 5/14
500/500 ━━━━━━━━━━━━━━━━━━━━ 68s 135ms/step - accuracy: 0.9733 - loss: 0.0793 - val_accuracy: 0.9605 - val_loss: 0.1061
Epoch 6/14
500/500 ━━━━━━━━━━━━━━━━━━━━ 67s 135ms/step - accuracy: 0.9781 - loss: 0.0622 - val_accuracy: 0.9695 - val_loss: 0.0735
Epoch 7/14
500/500 ━━━━━━━━━━━━━━━━━━━━ 67s 134ms/step - accuracy: 0.9883 - loss: 0.0395 - val_accuracy: 0.9685 - val_loss: 0.0822
Epoch 8/14
500/500 ━━━━━━━━━━━━━━━━━━━━ 67s 134ms/step - accuracy: 0.9903 - loss: 0.0306 - va

(0.04754861444234848, 0.9804999828338623)

In [23]:
mobilenet_model.save("mobilenet_best.h5")

In [16]:
# =======================
# Cell 1: Imports + generators for InceptionV3
# =======================

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models, optimizers
import tensorflow as tf

# InceptionV3 expects inputs scaled to [-1, 1]
inception_pre = tf.keras.applications.inception_v3.preprocess_input

# Use your existing helper to build train/val/test generators
train_inc, val_inc, test_inc = build_generators(
    TARGET_ROOT,
    preprocess_func=inception_pre
)

print(train_inc.samples, "train images")
print(val_inc.samples, "val images")
print(test_inc.samples, "test images")


Found 16000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
16000 train images
2000 val images
2000 test images


In [17]:
# =======================
# Cell 2: InceptionV3 – build model & Phase 1 (head only, 8 epochs)
# =======================

# 1) Base InceptionV3 (no top)
base_inception = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(256, 256, 3)
)

# Phase 1: freeze backbone, train only new head
base_inception.trainable = False

inputs = layers.Input(shape=(256, 256, 3))
x = base_inception(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)            # regularization similar to paper-style
outputs = layers.Dense(1, activation="sigmoid")(x)

inception_model = models.Model(inputs, outputs, name="InceptionV3_model")

# Slightly higher LR since only the head is trainable
inception_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print("=== Phase 1: InceptionV3 head training (8 epochs) ===")
inception_model.summary()

# Train head for 8 epochs
history_inc_p1, _ = train_and_evaluate_no_es(
    inception_model,
    train_inc,
    val_inc,
    test_inc,
    epochs=8,                # Phase 1: 8 epochs
    model_name="InceptionV3_phase1"
)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
=== Phase 1: InceptionV3 head training (8 epochs) ===


Model: "InceptionV3_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (None, 6, 6, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,804,833 (83.18 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

Epoch 1/8


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


500/500 ━━━━━━━━━━━━━━━━━━━━ 75s 117ms/step - accuracy: 0.6072 - loss: 0.6690 - val_accuracy: 0.7150 - val_loss: 0.5606
Epoch 2/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 93ms/step - accuracy: 0.6982 - loss: 0.5780 - val_accuracy: 0.7375 - val_loss: 0.5335
Epoch 3/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 46s 93ms/step - accuracy: 0.7097 - loss: 0.5620 - val_accuracy: 0.7395 - val_loss: 0.5318
Epoch 4/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 93ms/step - accuracy: 0.7258 - loss: 0.5441 - val_accuracy: 0.7575 - val_loss: 0.5025
Epoch 5/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 46s 93ms/step - accuracy: 0.7281 - loss: 0.5457 - val_accuracy: 0.7570 - val_loss: 0.5041
Epoch 6/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 93ms/step - accuracy: 0.7304 - loss: 0.5432 - val_accuracy: 0.7605 - val_loss: 0.4995
Epoch 7/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 93ms/step - accuracy: 0.7241 - loss: 0.5395 - val_accuracy: 0.7420 - val_loss: 0.5237
Epoch 8/8
500/500 ━━━━━━━━━━━━━━━━━━━━ 47s 93ms/step - accuracy: 0.7274 - loss: 0.5397 - val_accuracy: 0.7

In [19]:
# =======================
# Cell 3: InceptionV3 – Phase 2 fine-tuning (26 epochs, total 34)
# =======================

# Unfreeze top part of the InceptionV3 backbone for fine-tuning
# We keep BatchNorm layers frozen for stability.
for layer in base_inception.layers[-80:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

# Recompile with a much smaller learning rate for fine-tuning
inception_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print("\n=== Phase 2: Fine-tuning InceptionV3 (max 26 epochs) ===")
for i, layer in enumerate(base_inception.layers[-90:]):
    print(i, layer.name, "trainable" if layer.trainable else "frozen")

# Now train for up to 26 more epochs → 8 + 26 = 34 total (paper)
history_inc_p2, test_metrics_inception = train_and_evaluate_no_es(
    inception_model,
    train_inc,
    val_inc,
    test_inc,
    epochs=26,               # Phase 2: 26 epochs → 34 total
    model_name="InceptionV3_finetuned"
)

test_metrics_inception


=== Phase 2: Fine-tuning InceptionV3 (max 26 epochs) ===
0 batch_normalization_63 frozen
1 batch_normalization_68 frozen
2 batch_normalization_69 frozen
3 activation_60 frozen
4 activation_63 frozen
5 activation_68 frozen
6 activation_69 frozen
7 mixed7 frozen
8 conv2d_74 frozen
9 batch_normalization_72 frozen
10 activation_72 trainable
11 conv2d_75 trainable
12 batch_normalization_73 frozen
13 activation_73 trainable
14 conv2d_72 trainable
15 conv2d_76 trainable
16 batch_normalization_70 frozen
17 batch_normalization_74 frozen
18 activation_70 trainable
19 activation_74 trainable
20 conv2d_73 trainable
21 conv2d_77 trainable
22 batch_normalization_71 frozen
23 batch_normalization_75 frozen
24 activation_71 trainable
25 activation_75 trainable
26 max_pooling2d_5 trainable
27 mixed8 trainable
28 conv2d_82 trainable
29 batch_normalization_80 frozen
30 activation_80 trainable
31 conv2d_79 trainable
32 conv2d_83 trainable
33 batch_normalization_77 frozen
34 batch_normalization_81 frozen
3

(0.35085606575012207, 0.9330000281333923)

In [28]:
# ============================
# DATA GENERATORS FOR CNN MODEL
# ============================

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image size (paper uses 256×256 for all models)
IMG_SIZE = (256, 256)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    TARGET_ROOT / "train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_gen = val_test_datagen.flow_from_directory(
    TARGET_ROOT / "val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

test_gen = val_test_datagen.flow_from_directory(
    TARGET_ROOT / "test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)


Found 16000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [29]:
from tensorflow.keras import layers, models, regularizers, optimizers, callbacks

weight_decay = 1e-4
input_shape = (256, 256, 3)

def build_custom_cnn():
    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(
        32, (3, 3), padding="same",
        kernel_regularizer=regularizers.l2(weight_decay)
    )(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 2
    x = layers.Conv2D(
        64, (3, 3), padding="same",
        kernel_regularizer=regularizers.l2(weight_decay)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 3
    x = layers.Conv2D(
        128, (3, 3), padding="same",
        kernel_regularizer=regularizers.l2(weight_decay)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Block 4
    x = layers.Conv2D(
        256, (3, 3), padding="same",
        kernel_regularizer=regularizers.l2(weight_decay)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # You can add a 5th block if you want slightly more capacity:
    # x = layers.Conv2D(
    #     256, (3, 3), padding="same",
    #     kernel_regularizer=regularizers.l2(weight_decay)
    # )(x)
    # x = layers.BatchNormalization()(x)
    # x = layers.Activation("relu")(x)
    # x = layers.MaxPooling2D((2, 2))(x)

    # Classifier head
    x = layers.Flatten()(x)
    x = layers.Dense(
        256,
        kernel_regularizer=regularizers.l2(weight_decay)
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(0.6)(x)   # stronger dropout than before

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = models.Model(inputs, outputs, name="CustomCNN_l2_bn")
    return model

custom_cnn = build_custom_cnn()
custom_cnn.summary()


Model: "CustomCNN_l2_bn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_21 (InputLayer)     │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_102 (Conv2D)             │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_99          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_99 (Activation)      │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_103 (Conv2D)             │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_100         │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_100 (Activation)     │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_104 (Conv2D)             │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_101         │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_101 (Activation)     │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_105 (Conv2D)             │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_102         │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_102 (Activation)     │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │    16,777,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_103         │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_103 (Activation)     │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 17,169,089 (65.49 MB)

 Trainable params: 17,167,617 (65.49 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [30]:
# Compile with a slightly smaller LR
custom_cnn.compile(
    optimizer=optimizers.Adam(learning_rate=5e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

early_stop = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=8,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=4,
    min_lr=1e-6,
    verbose=1
)

history_cnn = custom_cnn.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,                 # max 40 like we discussed
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 238s 451ms/step - accuracy: 0.6170 - loss: 0.8885 - val_accuracy: 0.6685 - val_loss: 0.7094 - learning_rate: 5.0000e-05
Epoch 2/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 223s 445ms/step - accuracy: 0.6887 - loss: 0.7123 - val_accuracy: 0.6730 - val_loss: 0.7672 - learning_rate: 5.0000e-05
Epoch 3/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 222s 443ms/step - accuracy: 0.7218 - loss: 0.6575 - val_accuracy: 0.7695 - val_loss: 0.5637 - learning_rate: 5.0000e-05
Epoch 4/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 222s 444ms/step - accuracy: 0.7594 - loss: 0.5972 - val_accuracy: 0.7725 - val_loss: 0.5712 - learning_rate: 5.0000e-05
Epoch 5/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 222s 443ms/step - accuracy: 0.7675 - loss: 0.5750 - val_accuracy: 0.7410 - val_loss: 0.6546 - learning_rate: 5.0000e-05
Epoch 6/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 221s 443ms/step - accuracy: 0.7828 - loss: 0.5558 - val_accuracy: 0.7930 - val_loss: 0.5233 - learning_rate: 5.0000e-05
Epoch 7/40
500/500 ━━━━━━━━━━━━━━━━━━━━ 

In [31]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

test_gen.reset()
y_prob = custom_cnn.predict(test_gen, verbose=1).ravel()
y_pred = (y_prob >= 0.5).astype(int)
y_true = test_gen.classes  # assumes standard Keras generator

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:")
print(cm)

print("\nClassification report:")
print(classification_report(y_true, y_pred, target_names=["fake", "real"]))


63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step
Confusion matrix:
[[933  67]
 [ 74 926]]

Classification report:
              precision    recall  f1-score   support

        fake       0.93      0.93      0.93      1000
        real       0.93      0.93      0.93      1000

    accuracy                           0.93      2000
   macro avg       0.93      0.93      0.93      2000
weighted avg       0.93      0.93      0.93      2000

